### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Uses Natural Language processing combined with REGEX to extract information from text
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/ If you run out of space in your linux directory the script will crash. There is a cache file written by selenium that may then need to be deleted to get the chrome driver working again. Follow the path in the error message and delete that cache directory

In [ ]:
pip install "numpy<2.0" 

In [ ]:
#pip install spacy
#pip install -U pip setuptools wheel
#pip install -U spacy

### Install the model at the command lines
small model
python -m spacy download en_core_web_sm

medium model
python -m spacy download en_core_web_md

large model
python -m spacy download en_core_web_lg

### create training set https://spacy.io/usage/training

This annotator works better https://agateteam.org/spacynerannotate/ combined with the DocBin code to create the .spacy file. 

https://spacy.io/usage/training#quickstart

python -m spacy init fill-config ./base_config.cfg ./config.cfg

This NER annotator worked better than the one above. Used the DocBin technique to convert to .spacy file. Spacy training command with config.cfg then worked.

Config.cfg needs to be created for the spacy command line training. 

Follow the instructions in the quickstart for the base_config.cfg file. Selected OS and clicked NER

https://spacy.io/usage/training#quickstart

Run the following at command line to create the config.cfg. 

python -m spacy init fill-config ./base_config.cfg ./config.cfg

Once the config file was done and spacy file was created run the following at command line to train the model

To debug the data:
python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./train.spacy

To debug the config file file:
python -m spacy train config.cfg --output ./output

or

python -m spacy train config.cfg 

train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

creates two directories model-best and model-last. I used model-best 

nlp1 = spacy.load(r"./output/model-best")


Notes
python -m spacy init fill-config ./base_config.cfg ./config.cfg

python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

https://spacy.io/usage/training#basics

In [1]:
#Always needed
import spacy
nlp = spacy.load("en_core_web_md")

/home/mcveigh/jup/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mcveigh/jup/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Train spacy if not already done. Using output https://arunmozhi.in/ner-annotator/


In [ ]:
from spacy.tokens import DocBin
import pandas as pd
import json
import os

### train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy 

### Main webscraper program start


In [10]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime
from nameparser import HumanName
import spacy
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")
from spacy.matcher import PhraseMatcher
from spacy import displacy

In [3]:
#test
text2 = ("The type strain is 2205BS29-5T (=LMG 33062T =KACC 23240T), which is isolated from a marine sponge, P. elegans, "
"collected from Beomseom in Jeju-Island, Republic of Korea."

"The DNA G+C content of strain 2205BS29-5T is 67.8%. The GenBank accession numbers for the 16S rRNA gene and" 
"whole-genome sequences of strain 2205BS29-5T are OQ569368 and JAVAMQ000000000, respectively.")

In [4]:
#test but need this later
nlp1 = spacy.load(r"./output/model-best") #load the best model
doc = nlp1(text2)
from spacy import displacy
displacy.render(doc, style="ent")

### Find URLS from saved email in html - save as from outlook in htm format. URLs are extracted and saved as input for selenium

In [11]:
input = (r'IJSEMemail29.htm')
output = (r'NameCheckweek29.xlsx')

In [12]:
#with open ('IJSEMemail1.htm', encoding = 'unicode_escape') as f:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')  
#print(soup.prettify())

In [13]:
urls = []
numberoflinks = 0
for url in soup.findAll( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
    #print(url)
print(numberoflinks)
#print(urls)
#remove_list = ['TandC','doi.org', 'myaccount']
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e and "join-the-society" not in e]


#print(urls)

40


In [9]:
#def find_strains(description):
    #strains = []
#    nlp1 = spacy.load(r"./output/model-best") #load the best model
#    doc = nlp1(description)
#    for entity in doc.ents:
#        if entity.text not in strains:
#            entity.text.encode('ascii', 'ignore').decode('unicode_escape')
#            strains.append(entity.text)
#    return strains

In [14]:
def find_strains(description):
    doc = nlp(description)
    nlp1 = spacy.load(r"./output/model-best") #load the best model
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrases = ['strain', 'Strain', 'strains', 'Strains']
    patterns = [nlp(description) for description in phrases]
    phrase_matcher.add('STRAIN', None, *patterns)
    #strains = []
               
    for sent in doc.sents:    
        for match_id, start, end in phrase_matcher(nlp(sent.text)):
            if nlp.vocab.strings[match_id] in ["STRAIN"]:
                print(sent.text)
                doc = nlp1(sent.text)
                displacy.render(doc, style="ent")
                for entity in doc.ents:
                    if entity.text not in strains:
                        entity.text.encode('ascii', 'ignore').decode('unicode_escape')
                        strains.append(entity.text)
    return strains

# Remove non-ASCII characters
def remove_non_ascii(text):
    return ''.join(char for char in text if ord(char) < 128)



### Main body - Selenium to extract data from html
### https://nameparser.readthedocs.io/en/latest/index.html to find the last name

In [17]:
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'Authority', 'DOI', 'URL'])
pd.set_option('display.max_columns', None)
combined_description = []
for url in urls:
    counter = 1
    strains = []
    accessions = []
    doi = []
    author = []
    date = []
    year = []
    author_raw = []
    author = []
    name1 = []
    name2 = []
    author_count = []
    authority = []
    description = None
    snumber = []
    #scrape_with_selenium(url, counter)
    #scrape_with_beautifulsoup(url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        #print(doi)
    
    #find publication year
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[3]/span/span[2]"):
        date = element.text
        year = date[-4:]
        #print(year)
    
    #find authors
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[1]/span"):
        author_raw = element.text
        #print(author_raw)
        author = re.sub(r"[\d+]+",'',author_raw)
        author = re.sub(r"†",'',author)
        author = re.sub(r" and ",',',author)
        author = re.sub(r",,",',',author)  
        author = author.split(',')
        author[:] = [item for item in author if item != '']
        #print(author)
        author_count = len(author)
        #print(author_count)
        if author_count == 1:
            name1 = (str(author[0])) 
            name1 = HumanName(name1)
            authority = name1.last + ' ' +str(year)
            #print(authority)
        elif author_count ==2:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            name2 = (str(author[1]))
            name2 = HumanName(name2)
            authority = name1.last + ' and ' + name2.last + ' ' +str(year)
            #print(authority)
        else:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            authority = name1.last + ' et al. ' +str(year)
            #print(authority)
    
    #extract data from the species description
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description: 
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                cleaned_text = remove_non_ascii(description)
                combined_description.append(cleaned_text)
                #print(cleaned_text)
                #description = description.replace('\u2009', ' ')
                #print(description)
        
                #find the organism names    
                match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                orgname = [m.group() for m in regex.finditer(description)]
                print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    print('accessions', accessions)
    
                #find the strains
                #strainpattern = [r'(?<=type strain).*'] 
                #strainpattern = [r'(?<=type strain).*?(?=\))']
                #regex = re.compile(r'\b(' + '|'.join(strainpattern) + r')\b')
                if description is not None:
                    #description = description.encode('ascii', 'ignore').decode('unicode_escape')
                    #description = description.replace('\u2009', ' ')
                    find_strains(cleaned_text)
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, authority, doi, url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK')
    
#Close the browser window
    driver.quit()    

Roseomonas cutis sp. nov., isolated from human epidermis
orgname ['Roseomonas cutis ']
accessions ['OP788068']
The type strain is OT10T (=KCTC 92087T =JCM 34968T), isolated from male skin swab sample in South Korea.


The DNA G+C content of the type strain is 72.6mol% (genome).


/home/mcveigh/jup/lib/python3.9/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


The GenBank accession numbers of the 16S rRNA gene and whole-genome sequences of strain OT10T are OP788068 and ASM2099108v1, respectively.
 


strain names ['OT10T', 'KCTC 92087T', 'JCM 34968T']
orgname ['Roseomonas cutis ']
accessions []
strain names ['OT10T', 'KCTC 92087T', 'JCM 34968T']
BREAK
Addition of Section 10, Rules 66–73 for further integration of Candidatus names into the International Code of Nomenclature of Prokaryotes
Identification of Starmerella aleppica f.a., sp. nov. and large indels in the rRNA cistron that split the Starmerella genus
orgname ['and Baghela ']
accessions ['MB855459']
The holotype CBS 12960 (original strain number 2-1361) was isolated from a Malva sylvestris flower in Syria in 2009 and is preserved in a metabolically inactive state in the CBS Yeast Collection of the Westerdijk Fungal Biodiversity Institute, Utrecht, the Netherlands.


...

The accession numbers of the genome of strain P. alni subsp.


The type strain is JCM 14827T (=


strain names ['DW16-2T', 'TBRC 16418T', 'NBRC 115857T', 'DSM 44104T', 'IFO 14991T', 'JCM 9103T', 'NBRC 14991T', 'VKM Ac-901T', 'JCM 14827T']
orgname ['Pseudonocardia alni subsp. carboxydivorans ']
accessions []
strain names ['DW16-2T', 'TBRC 16418T', 'NBRC 115857T', 'DSM 44104T', 'IFO 14991T', 'JCM 9103T', 'NBRC 14991T', 'VKM Ac-901T', 'JCM 14827T']
BREAK
Genome-based reclassification of Suonthocola fibrivorans as a later heterotypic synonym of Diplocloster agilis
Vishniacozyma siamensis sp. nov., a new anamorphic tremellomycetous yeast species isolated from a mangrove forest in Thailand
orgname ['and Khunnamwong ', 'Vishniacozyma siamensis ', 'V. siamensis ']
accessions ['LC786857', 'LC832838', 'LC832837', 'LC832840', 'LC832839', 'LC818861', 'LC797015', 'LC832841', 'LC797016', 'LC832842', 'LC832843', 'LC832855']
adj. siamensis, of or belonging to Siam, the old name of Thailand, where the type strain was isolated).



/home/mcveigh/jup/lib/python3.9/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


strain names []
orgname ['and Khunnamwong ']
accessions []
strain names []
BREAK


In [18]:
#optional write description to a file
#print(combined_description)
file = open('all_descriptions29', "w")
file.writelines(combined_description)
file.close()

In [19]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [', '.join(map(str, l)) for l in pub_df['Strains']]
#pub_df['Strains'] = pub_df['Strains'].astype(str) 
pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
#pub_df
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
#pub_df
#try drop duplicate accessions here
pub_df.explode(['PublishedName']).reset_index(drop=True)
pub_df

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,[Roseomonas cutis ],[OP788068],"OT10T, KCTC 92087T, JCM 34968T",Son et al. 2025,https://doi.org/10.1099/ijsem.0.006617,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006617%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018546452%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=qPgbRuHApRO85zY1%2FXyHK7fwpRhZIQDtoO%2BMyTwMQ8I%3D&reserved=0
2,[and Baghela ],[MB855459],"CBS 12960, 2-1361",Sipiczki and Baghela 2025,https://doi.org/10.1099/ijsem.0.006629,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006629%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018597448%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=gTvoT9taoY5EywTBMznivistMD3RQuxyiE0TB92Vbu8%3D&reserved=0
4,[Methylobacterium litchii ],"[PP600109, JBBXMY000000000]","SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
6,[Methylobacterium guangdongense ],"[PP600110, JBBXMX000000000]","SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
8,[Methylorubrum subtropicum ],"[PP600111, JBBXMW000000000]","SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
10,"[Pseudomonas fungipugnans , P. fungipugnans ]","[OQ547267, JARBWL000000000]","DSM 115721, LMG 33039",Schnyder et al. 2025,https://doi.org/10.1099/ijsem.0.006624,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006624%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018675073%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpb

In [133]:
#pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
#pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
#pub_df

In [20]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
#pub2_df
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
#pub4_df

In [21]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique = df_unique.dropna()
#df_unique

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [22]:
df_unique['accession'] = df_unique['accession'].astype('str') 

/tmp/ipykernel_3550981/1867462633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['accession'] = df_unique['accession'].astype('str')


In [23]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [24]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


Error: Malformatted ID "nan"


0

In [25]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna()
srcchk_df 
#many need to add dropna here
#If /strain is not populated on the NCBI accession, we lose the NCBI srcchk info here

,accession,NCBIname,taxid,strain
0,OP788068,Roseomonas sp. OT10,2897332.0,OT10
2,PP600109,Methylobacterium sp.,409.0,SD21
3,JBBXMY000000000,Methylobacterium sp. SD21,3138810.0,SD21
4,PP600110,Methylobacterium sp.,409.0,SI9
5,JBBXMX000000000,Methylobacterium sp. SI9,3138811.0,SI9
6,PP600111,Methylorubrum sp.,2282524.0,SB2
7,JBBXMW000000000,Methylorubrum sp. SB2,3138812.0,SB2
8,OQ547267,Pseudomonas sp.,306.0,#681
9,JARBWL000000000,Pseudomonas sp. 681,3024217.0,681
11,PQ066384,Leptospira sp. WS58.C1,3235173.0,WS58.C1


### Combine dataframes into one

In [26]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'Authority', 'taxid', 'DOI', 'URL' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,Roseomonas cutis,Roseomonas sp. OT10,"OT10T, KCTC 92087T, JCM 34968T",OP788068,OT10,Son et al. 2025,2897332.0,https://doi.org/10.1099/ijsem.0.006617,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006617%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018546452%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=qPgbRuHApRO85zY1%2FXyHK7fwpRhZIQDtoO%2BMyTwMQ8I%3D&reserved=0
1,and Baghela,NaN,"CBS 12960, 2-1361",MB855459,NaN,Sipiczki and Baghela 2025,NaN,https://doi.org/10.1099/ijsem.0.006629,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006629%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018597448%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=gTvoT9taoY5EywTBMznivistMD3RQuxyiE0TB92Vbu8%3D&reserved=0
2,Methylobacterium litchii,Methylobacterium sp.,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",PP600109,SD21,Gao et al. 2025,409.0,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
3,Methylobacterium litchii,Methylobacterium sp. SD21,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",JBBXMY000000000,SD21,Gao et al. 2025,3138810.0,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
4,Methylobacterium guangdongense,Methylobacterium sp.,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",PP600110,SI9,Gao et al. 2025,409.0,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
5,Methylobacterium guangdongense,Methylobacterium sp. SI9,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",JBBXMX000000000,SI9,Gao et al. 2025,3138811.0,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b775789

In [27]:
def highlight_rows(row):
    ijsemvalue = row.loc['PublishedName']
    ncbivalue = row.loc['NCBIname']
    if ijsemvalue != ncbivalue:
        color = '#FFB3BA' # Red
    elif ijsemvalue == ncbivalue:
        color = '#BAFFC9' # Green
    return ['background-color: {}'.format(color) for r in row]

new_df = combine_df.style.apply(highlight_rows, axis=1, subset=['PublishedName', 'NCBIname'])
new_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,Roseomonas cutis,Roseomonas sp. OT10,"OT10T, KCTC 92087T, JCM 34968T",OP788068,OT10,Son et al. 2025,2897332.000000,https://doi.org/10.1099/ijsem.0.006617,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006617%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018546452%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=qPgbRuHApRO85zY1%2FXyHK7fwpRhZIQDtoO%2BMyTwMQ8I%3D&reserved=0
1,and Baghela,nan,"CBS 12960, 2-1361",MB855459,nan,Sipiczki and Baghela 2025,nan,https://doi.org/10.1099/ijsem.0.006629,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006629%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018597448%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=gTvoT9taoY5EywTBMznivistMD3RQuxyiE0TB92Vbu8%3D&reserved=0
2,Methylobacterium litchii,Methylobacterium sp.,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",PP600109,SD21,Gao et al. 2025,409.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
3,Methylobacterium litchii,Methylobacterium sp. SD21,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",JBBXMY000000000,SD21,Gao et al. 2025,3138810.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
4,Methylobacterium guangdongense,Methylobacterium sp.,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",PP600110,SI9,Gao et al. 2025,409.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
5,Methylobacterium guangdongense,Methylobacterium sp. SI9,"SD21T, GDMCC 1.4327T, KCTC 8300T, SI9T, SB2T, GDMCC 1.14328T=, KCTC 8299T",JBBXMX000000000,SI9,Gao et al. 2025,3138811.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8

### write output to excel

In [28]:
new_df.to_excel(output, engine='xlsxwriter', index = False, na_rep = '') 

In [116]:
combine_df.dtypes

PublishedName            object
NCBIname                 object
Strains          string[python]
accession                object
strain                   object
Authority                object
taxid                   float64
DOI                      object
URL                      object
dtype: object